# Setup Trustee

Since we still are in the user secure environment, start Trustee server using a prepared container image at `quay.io/confidential-devhub/trustee-server` with the right parameters.

## Create kbs-config.toml

Set up basic configs like the http(s) server, which key to trust and so on. Refer to the [upstream documentation](https://github.com/confidential-containers/trustee/blob/main/kbs/docs/config.md) for more info about kbs-config.toml.

In [ ]:
%%bash

MACHINE_IP_ADDR=<your_trustee_ip_address>

cat > kbs-config.toml <<EOF
[http_server]
sockets = ["127.0.0.1:8080","$MACHINE_IP_ADDR:8080"]
insecure_http = true

[admin]
insecure_api = true

[attestation_token]
insecure_key = true
attestation_token_type = "CoCo"

[attestation_service]
type = "coco_as_builtin"
work_dir = "/opt/confidential-containers/attestation-service"
policy_engine = "opa"

  [attestation_service.attestation_token_broker]
  type = "Ear"
  policy_dir = "/opt/confidential-containers/attestation-service/policies"

  [attestation_service.attestation_token_config]
  duration_min = 5

  [attestation_service.rvps_config]
  type = "BuiltIn"

    [attestation_service.rvps_config.storage]
    type = "LocalJson"
    file_path = "/opt/confidential-containers/rvps/reference-values/reference-values.json"

[[plugins]]
name = "resource"
type = "LocalFs"
dir_path = "/opt/confidential-containers/kbs/repository"

[policy_engine]
policy_path = "/opt/confidential-containers/opa/policy.rego"

EOF

## Optional configs

For example, the Trustee server defaults to empty reference values `[]`. By creating `reference-values.json` we can provide actual expected measurements. To keep this demo simple, we will not provide them.

In [ ]:
%%bash

podman run \
  --privileged \
  -v ./kbs-config.toml:/etc/kbs-config/kbs-config.toml \
  -v ./auth_public.pub:/etc/auth-secret/publicKey \
  -v /dev/log:/dev/log \
  -p 8080:8080 \
  --rm \
  quay.io/confidential-devhub/trustee-server:v0.3.0 \
  /usr/local/bin/kbs --config-file /etc/kbs-config/kbs-config.toml

## Run the Trustee server

Run the container image `quay.io/confidential-devhub/trustee-server`.

Insert `key.bin` as secret in Trustee located at path `default/fraud-detection/key.bin`.

Note that in this example we won't use `https`.
If the https self signed certificate has been added, extend the podman command with
```
  -v $HTTPS_PRIVATE_KEY:/https_private_key \
  -v $HTTPS_CERT:/https_certificate \
```

If the reference values are provided, extend the podman command with
```
-v $REFERENCE_VALUES:/opt/confidential-containers/rvps/reference-values/reference-values.json
```

In [ ]:
%%bash

DECRYPTION_KEY=./key.bin
KBS_RETRIEVAL_PATH=fraud-detection/key.bin
MACHINE_IP_ADDR=<your_trustee_ip_address> # Insert value here



KBS_CONFIG=./kbs-config.toml # insert kbs-config path
DECRYPTION_KEY=./key.bin # insert decryption key path

TRUSTEE_SECRETS_PATH=/opt/confidential-containers/kbs/repository/default
KBS_RETRIEVAL_PATH=fraud-detection/key.bin # insert custom path here

podman run \
  --privileged \
  -v $KBS_CONFIG:/etc/kbs-config/kbs-config.toml \
  -v $DECRYPTION_KEY:$TRUSTEE_SECRETS_PATH/$KBS_RETRIEVAL_PATH
  -v /dev/log:/dev/log \
  -p 8080:8080 \
  --rm \
  quay.io/confidential-devhub/trustee-server:v0.3.0 \
  /usr/local/bin/kbs --config-file /etc/kbs-config/kbs-config.toml

The output should be

```
[2025-04-24T07:54:20Z INFO  kbs] Using config file /etc/kbs-config/kbs-config.toml
[2025-04-24T07:54:20Z WARN  kbs::admin] insecure admin APIs are enabled
[2025-04-24T07:54:20Z INFO  attestation_service::rvps] launch a built-in RVPS.
[2025-04-24T07:54:20Z INFO  attestation_service::token::ear_broker] Loading default AS policy "ear_default_policy.rego"
[2025-04-24T07:54:20Z INFO  attestation_service::token::ear_broker] No Token Signer key in config file, create an ephemeral key and without CA pubkey cert
[2025-04-24T07:54:20Z INFO  kbs::api_server] Starting HTTP server at [127.0.0.1:8080, 10.0.0.5:8080]
[2025-04-24T07:54:20Z INFO  actix_server::builder] starting 2 workers
[2025-04-24T07:54:20Z INFO  actix_server::server] Tokio runtime found; starting in existing Tokio runtime
[2025-04-24T07:54:20Z INFO  actix_server::server] starting service: "actix-web-service-127.0.0.1:8080", workers: 2, listening on: 127.0.0.1:8080
[2025-04-24T07:54:20Z INFO  actix_server::server] starting service: "actix-web-service-10.0.0.5:8080", workers: 2, listening on: 10.0.0.5:8080
 ```